Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [25]:
import tensorflow as tf
import os
import zipfile

In [26]:
DESIRED_ACCURACY = 0.999

zip_ref = zipfile.ZipFile("./data/happy-or-sad.zip", 'r')
zip_ref.extractall("./data/h-or-s")
zip_ref.close()

In [27]:
class EarlyStopCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc') > DESIRED_ACCURACY ):
            print("/nReached {}% accuracy so cancelling training! - {}".format(DESIRED_ACCURACY, logs.get('acc')))
            self.model.stop_training=True

callback = EarlyStopCallBack()

In [28]:
train_happy_dir = os.path.join('./data/h-or-s/happy')
train_sad_dir = os.path.join('./data/h-or-s/sad')

In [32]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [33]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    './data/h-or-s',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [34]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1, 
    callbacks=[callback])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 1s 109ms/step - loss: 0.8371 - acc: 0.5250
Epoch 2/15
8/8 [==============================] - 1s 80ms/step - loss: 0.6465 - acc: 0.6250
Epoch 3/15
8/8 [==============================] - 1s 88ms/step - loss: 0.6133 - acc: 0.7625
Epoch 4/15
8/8 [==============================] - 1s 78ms/step - loss: 0.3877 - acc: 0.8250
Epoch 5/15
8/8 [==============================] - 1s 78ms/step - loss: 0.2858 - acc: 0.8500
Epoch 6/15
8/8 [==============================] - 1s 96ms/step - loss: 0.2475 - acc: 0.8875
Epoch 7/15
8/8 [==============================] - 1s 80ms/step - loss: 0.1757 - acc: 0.9125
Epoch 8/15
8/8 [==============================] - 1s 77ms/step - loss: 0.1889 - acc: 0.9125
Epoch 9/15
8/8 [==============================] - 1s 82ms/step - loss: 0.1266 - acc: 0.9375
Epoch 10/15
8/8 [==============================] - 1s 79ms/step - loss: 0.0214 - acc: 1.0000
